# Analiza rynku Airbnb w Winnipeg
## Projekt zaliczeniowy - Analiza danych w języku R

---

**Autor:** Karol Wilczyński  
**Data:** 14 czerwca 2025  
**Przedmiot:** Analiza danych w języku R  

---

## Spis treści

1. [Wprowadzenie](#wprowadzenie)
2. [Załadowanie danych](#załadowanie-danych)
3. [Rozmiar zbioru danych](#rozmiar-zbioru-danych)
4. [Eksploracja surowych danych](#eksploracja-surowych-danych)
5. [Weryfikacja typów zmiennych](#weryfikacja-typów-zmiennych)
6. [Podsumowanie zmiennych](#podsumowanie-zmiennych)
7. [Analiza braków danych](#analiza-braków-danych)
8. [Wizualizacja rozkładów](#wizualizacja-rozkładów)
9. [Czyszczenie danych](#czyszczenie-danych)
10. [Analiza zależności](#analiza-zależności)
11. [Analiza dodatkowa](#analiza-dodatkowa)
12. [Podsumowanie i wnioski](#podsumowanie-i-wnioski)

---

## 1. Wprowadzenie

### Cel analizy
Celem niniejszego projektu jest kompleksowa analiza rynku wynajmu krótkoterminowego Airbnb w Winnipeg, stolicy prowincji Manitoba w Kanadzie. Analiza obejmuje wszystkie wymagane etapy przetwarzania i eksploracji danych, od załadowania surowych danych po badanie złożonych zależności między zmiennymi.

![braki](https://drive.google.com/uc?export=download&id=1mUzR_SrIhYeYVRCUoKYs1-aJe9aziU0M)

### Źródło danych
Dane pochodzą z platformy Inside Airbnb (http://insideairbnb.com/get-the-data.html) i zawierają szczegółowe informacje o ofertach wynajmu w Winnipeg z dnia 7 marca 2025 roku.

### Narzędzia
Analiza została przeprowadzona w środowisku R z wykorzystaniem następujących głównych pakietów:
- **tidyverse** - przetwarzanie i wizualizacja danych
- **mice** - imputacja braków danych
- **corrplot** - wizualizacja korelacji
- **leaflet** - mapy interaktywne
- **psych** - analiza statystyczna

---

## 2. Załadowanie danych

### Kod załadowania

```r
# Instalacja i ładowanie wymaganych pakietów
install.packages(c(
  "tidyverse", "lubridate", "corrplot", "VIM", "mice",
  "gridExtra", "scales", "psych", "car", "GGally",
  "leaflet", "htmlwidgets", "tidytext", "ggrepel"
), repos = "http://cran.us.r-project.org")

# Ładowanie bibliotek
suppressPackageStartupMessages({
  library(tidyverse)    
  library(lubridate)    
  library(corrplot)     
  library(VIM)          
  library(mice)         
  library(gridExtra)    
  library(scales)       
  library(psych)        
  library(car)          
  library(GGally)       
  library(leaflet)      
  library(htmlwidgets)  
  library(tidytext)     
  library(ggrepel)      
})

# URL do danych Winnipeg
url <- "https://data.insideairbnb.com/canada/mb/winnipeg/2025-03-07/data/listings.csv.gz"

# Załadowanie danych
df <- read_csv(
  url,
  col_types = cols(),
  locale = locale(encoding = "UTF-8"),
  na = c("", "NA", "N/A", "null"),
  show_col_types = FALSE
)
```

### Wynik
✓ Dane załadowane pomyślnie!  
✓ Źródło: Winnipeg, Manitoba, Kanada  

![braki](https://drive.google.com/uc?export=download&id=1F-9lh3v8Fsea6uWnU158u0OwmP8Y9NAQ)

---

## 3. Rozmiar zbioru danych

```r
# Podstawowe informacje o zbiorze
n_rows <- nrow(df)
n_cols <- ncol(df)
memory_size <- object.size(df)

cat("Liczba obserwacji (wierszy):", format(n_rows, big.mark = ","), "\n")
cat("Liczba zmiennych (kolumn):", n_cols, "\n")
cat("Rozmiar w pamięci:", format(memory_size, units = "MB"), "\n")
```

### Wyniki:
- **Liczba obserwacji (wierszy):** 1,626
- **Liczba zmiennych (kolumn):** 79
- **Rozmiar w pamięci:** 4.2 MB

### Oszacowanie czasochłonności
| Operacja | Czas próbki (s) | Szacowany czas pełny (s) |
|----------|-----------------|--------------------------|
| Statystyki opisowe | 0.025 | 0.41 |
| Macierz korelacji | 0.069 | 1.12 |
| Grupowanie danych | 0.010 | 0.16 |
| Tworzenie wykresu | 0.006 | 0.10 |

✓ Analiza jest wykonalna - szacowany całkowity czas < 2 sekundy

---

## 4. Eksploracja surowych danych

```r
# Lista wszystkich kolumn
cat("\nWSZYSTKIE KOLUMNY (", n_cols, "):\n", sep = "")
cat(paste(names(df), collapse = ", "), "\n")

# Wyświetlenie próbki danych
key_columns <- c("id", "name", "price", "room_type", "property_type",
                "accommodates", "bedrooms", "beds", "bathrooms_text",
                "neighbourhood_cleansed", "review_scores_rating",
                "host_is_superhost", "minimum_nights")

available_key_cols <- key_columns[key_columns %in% names(df)]
print(head(df[, available_key_cols], 5))
```

### Lista kolumn (79):
id, listing_url, scrape_id, last_scraped, source, name, description, neighborhood_overview, picture_url, host_id, host_url, host_name, host_since, host_location, host_about, host_response_time, host_response_rate, host_acceptance_rate, host_is_superhost, host_thumbnail_url, host_picture_url, host_neighbourhood, host_listings_count, host_total_listings_count, host_verifications, host_has_profile_pic, host_identity_verified, neighbourhood, neighbourhood_cleansed, neighbourhood_group_cleansed, latitude, longitude, property_type, room_type, accommodates, bathrooms, bathrooms_text, bedrooms, beds, amenities, price, minimum_nights, maximum_nights, minimum_minimum_nights, maximum_minimum_nights, minimum_maximum_nights, maximum_maximum_nights, minimum_nights_avg_ntm, maximum_nights_avg_ntm, calendar_updated, has_availability, availability_30, availability_60, availability_90, availability_365, calendar_last_scraped, number_of_reviews, number_of_reviews_ltm, number_of_reviews_l30d, availability_eoy, number_of_reviews_ly, estimated_occupancy_l365d, estimated_revenue_l365d, first_review, last_review, review_scores_rating, review_scores_accuracy, review_scores_cleanliness, review_scores_checkin, review_scores_communication, review_scores_location, review_scores_value, license, instant_bookable, calculated_host_listings_count, calculated_host_listings_count_entire_homes, calculated_host_listings_count_private_rooms, calculated_host_listings_count_shared_rooms, reviews_per_month

### Próbka danych (pierwsze 5 wierszy):
| id | name | price | room_type | accommodates | bedrooms |
|----|------|-------|-----------|--------------|----------|
| 1526583 | Cozy and friendly home! | 48.00 | Private room | 1 | 3 |
| 1529013 | Cosy! Private! Wolseley! | 63.00 | Entire home/apt | 2 | 0 |
| 1559076 | Executive Penthouse Condo | 72.00 | Entire home/apt | 2 | 1 |
| 1664166 | Contemporary 5 bedroom | 186.00 | Entire home/apt | 10 | 5 |
| 3664955 | Heart of the city | 131.00 | Entire rental unit | 4 | 2 |

---

## 5. Weryfikacja typów zmiennych

```r
# Sprawdzenie typów przed konwersją
types_before <- df %>%
  summarise_all(~class(.)[1]) %>%
  gather(key = "Kolumna", value = "Typ") %>%
  count(Typ) %>%
  arrange(desc(n))

print(types_before)
```

### Typy przed konwersją
| Typ | Liczba zmiennych |
|-----|------------------|
| numeric | 42 |
| character | 25 |
| logical | 7 |
| Date | 5 |

```r
# Konwersja typów zmiennych
df <- df %>%
  mutate(
    # NUMERYCZNE - konwersja cen
    price = as.numeric(gsub("[$,]", "", price)),
    
    # KATEGORYCZNE
    room_type = as.factor(room_type),
    property_type = as.factor(property_type),
    neighbourhood_cleansed = as.factor(neighbourhood_cleansed),
    
    # NUMERYCZNE z tekstu
    bathrooms_numeric = as.numeric(str_extract(bathrooms_text, "\\d+\\.?\\d*"))
  )
```

### Typy po konwersji
| Typ | Liczba zmiennych |
|-----|------------------|
| numeric | 46 |
| character | 18 |
| logical | 7 |
| Date | 5 |
| factor | 3 |
| ordered | 1 |

✓ Konwersja typów zakończona pomyślnie!

---

## 6. Podsumowanie zmiennych

### Zmienne numeryczne

```r
# Podsumowanie zmiennych numerycznych
numeric_vars <- c("price", "accommodates", "bedrooms", "beds", "bathrooms_numeric",
                 "minimum_nights", "maximum_nights", "number_of_reviews",
                 "reviews_per_month", "availability_365", "review_scores_rating")

numeric_summary <- df %>%
  select(all_of(numeric_vars)) %>%
  summarise_all(list(
    Min = ~min(., na.rm = TRUE),
    Max = ~max(., na.rm = TRUE),
    Srednia = ~mean(., na.rm = TRUE),
    Mediana = ~median(., na.rm = TRUE),
    Q1 = ~quantile(., 0.25, na.rm = TRUE),
    Q3 = ~quantile(., 0.75, na.rm = TRUE),
    SD = ~sd(., na.rm = TRUE),
    Braki = ~sum(is.na(.))
  ))
```

| Zmienna | Min | Max | Średnia | Mediana | Q1 | Q3 | SD | Braki |
|---------|-----|-----|---------|---------|----|----|----|----|
| price | 20.00 | 6000.00 | 121.61 | 89.00 | 64.00 | 124.00 | 276.55 | 173 |
| accommodates | 1.00 | 16.00 | 3.64 | 4.00 | 2.00 | 4.00 | 2.10 | 0 |
| bedrooms | 0.00 | 6.00 | 1.77 | 2.00 | 1.00 | 2.00 | 0.97 | 46 |
| beds | 0.00 | 12.00 | 1.92 | 2.00 | 1.00 | 2.00 | 1.20 | 176 |
| bathrooms_numeric | 0.00 | 5.00 | 1.29 | 1.00 | 1.00 | 1.50 | 0.58 | 5 |
| minimum_nights | 1.00 | 365.00 | 8.85 | 2.00 | 1.00 | 4.00 | 25.99 | 0 |
| number_of_reviews | 0.00 | 844.00 | 47.16 | 19.00 | 3.00 | 55.00 | 78.86 | 0 |
| review_scores_rating | 1.00 | 5.00 | 4.77 | 4.85 | 4.71 | 4.96 | 0.34 | 234 |
| availability_365 | 0.00 | 365.00 | 198.10 | 225.00 | 65.00 | 341.00 | 138.61 | 0 |

### Zmienne kategoryczne

**room_type:**
- Entire home/apt: 1177 (72.4%)
- Private room: 449 (27.6%)

**property_type:**
- Top 5 kategorii:
  - Entire home: 539 (33.1%)
  - Private room in home: 346 (21.3%)
  - Entire rental unit: 239 (14.7%)
  - Entire condo: 175 (10.8%)
  - Entire guest suite: 116 (7.1%)

**neighbourhood_cleansed:**
- Top 5 dzielnic:
  - Fort Rouge - East Fort Garry: 228 (14.0%)
  - Waverley West: 227 (14.0%)
  - St. Vital: 181 (11.1%)
  - Daniel McIntyre: 170 (10.5%)
  - Point Douglas: 121 (7.4%)

**Superhości:**
- Superhostów: 643 (40.5%)
- Zwykłych hostów: 945 (59.5%)

---

## 7. Analiza braków danych

```r
# Obliczenie braków dla wszystkich zmiennych
missing_analysis <- df %>%
  summarise_all(~sum(is.na(.))) %>%
  gather(key = "Zmienna", value = "Liczba_brakow") %>%
  mutate(
    Procent_brakow = round(Liczba_brakow / n_rows * 100, 2)
  ) %>%
  arrange(desc(Procent_brakow))
```

### Podsumowanie braków
- Zmienne bez braków: 50
- Zmienne z brakami: 30
- Zmienne z brakami >50%: 6
- Zmienne z brakami 20-50%: 1
- Zmienne z brakami <20%: 23

### Wykres braków danych
![braki](https://drive.google.com/uc?export=download&id=1b9ZYMlaVYKdb6b7LJynNTvZ96NzRIbpH)

### TOP 10 zmiennych z największymi brakami

| Zmienna | Liczba braków | Procent braków |
|---------|---------------|----------------|
| neighbourhood_group_cleansed | 1626 | 100.00% |
| calendar_updated | 1626 | 100.00% |
| license | 945 | 58.12% |
| host_about | 893 | 54.92% |
| neighborhood_overview | 876 | 53.87% |
| neighbourhood | 876 | 53.87% |
| host_location | 451 | 27.73% |
| host_response_time | 244 | 15.00% |
| host_response_rate | 244 | 15.00% |
| reviews_per_month | 234 | 14.39% |

### Prawdopodobne przyczyny braków
- **license (58%)** - pole opcjonalne, nie wszystkie miasta wymagają licencji
- **host_about (55%)** - profil opcjonalny wypełniany przez gospodarzy
- **review_scores_* (14%)** - brak gdy nieruchomość nie ma jeszcze recenzji

---

## 8. Wizualizacja rozkładów

### Charakterystyki rozkładów zmiennych numerycznych

```r
# Analiza skośności i kurtozy
distribution_stats <- df %>%
  select(price, accommodates, bedrooms, minimum_nights,
         number_of_reviews, review_scores_rating) %>%
  summarise_all(list(
    skewness = ~psych::skew(.),
    kurtosis = ~psych::kurtosi(.)
  ))
```

| Zmienna | Skośność | Interpretacja skośności | Kurtoza | Interpretacja kurtozy |
|---------|----------|------------------------|---------|---------------------|
| price | 16.246 | prawostronnie skośny | 292.774 | leptokurtyczny (wysmukły) |
| accommodates | 1.473 | prawostronnie skośny | 3.943 | leptokurtyczny (wysmukły) |
| bedrooms | 1.226 | prawostronnie skośny | 1.729 | leptokurtyczny (wysmukły) |
| minimum_nights | 10.126 | prawostronnie skośny | 128.141 | leptokurtyczny (wysmukły) |
| number_of_reviews | 3.807 | prawostronnie skośny | 21.358 | leptokurtyczny (wysmukły) |
| review_scores_rating | -5.768 | lewostronnie skośny | 50.600 | leptokurtyczny (wysmukły) |

### Wykresy rozkładów
![Rozkłady kluczowych zmiennych numerycznych](https://drive.google.com/uc?export=download&id=1-vueG_mLQERbldKRzDArD_EwrvPWpQQP)

### Struktura rynku

#### Rozkład typów pokoi i superhostów
![Rozkład typów pokoi i superhostów](https://drive.google.com/uc?export=download&id=1ND9W6y3CJJISAn-Fxm9emrpI0U6uxDpU)

#### Analiza multi-listings

```r
# Oblicz liczbę ofert na gospodarza
host_listings <- df %>%
  group_by(host_id, host_name) %>%
  summarise(
    liczba_ofert = n(),
    srednia_cena = mean(price, na.rm = TRUE),
    .groups = 'drop'
  ) %>%
  arrange(desc(liczba_ofert))
```

### Struktura gospodarzy
- Gospodarze z 1 ofertą: 656 (70.8%)
- Gospodarze z wieloma ofertami: 271 (29.2%)
- Łączna liczba multi-listings: 970 (59.7% wszystkich ofert)

![Rozkład liczby ofert na gospodarza](https://drive.google.com/uc?export=download&id=1wB8E2tNbqXI7MtKopbr43YoTMAdAzJVQ)

![Udział w rynku według liczby ofert](https://drive.google.com/uc?export=download&id=1mGPyWSGiXaJ01oj5hNvKbwscZuhlPxSk)

### TOP 10 gospodarzy

| Nazwa gospodarza | Liczba ofert | Średnia cena |
|-----------------|--------------|--------------|
| Nest Host | 55 | 128.00 |
| Krystin | 43 | 132.00 |
| Luxxe Host | 25 | 104.00 |
| Curtis | 20 | 91.20 |
| E Street | 19 | 112.00 |
| Murray And Carley | 13 | 155.00 |
| Vladimir | 12 | 103.00 |
| Andrea | 10 | 52.10 |
| Brayden | 10 | 52.00 |
| Stas | 9 | 151.00 |

---

## 9. Czyszczenie danych

```r
# KROK 1: Usunięcie kolumn z >70% braków
cols_to_remove <- missing_analysis %>%
  filter(Procent_brakow > 70) %>%
  pull(Zmienna)

df <- df %>% select(-all_of(cols_to_remove))

# KROK 2: Usunięcie wierszy bez ceny
df <- df %>% filter(!is.na(price))

# KROK 3: Przycięcie wartości odstających
trim_outliers <- function(x, lower_percentile = 0.01, upper_percentile = 0.99) {
  bounds <- quantile(x, c(lower_percentile, upper_percentile), na.rm = TRUE)
  x[x < bounds[1]] <- bounds[1]
  x[x > bounds[2]] <- bounds[2]
  return(x)
}

df$price <- trim_outliers(df$price)
df$minimum_nights <- trim_outliers(df$minimum_nights)
df$maximum_nights <- trim_outliers(df$maximum_nights)
```

### Proces czyszczenia

#### KROK 1: Usuwanie kolumn z >70% braków
Usunięte kolumny: neighbourhood_group_cleansed, calendar_updated

#### KROK 2: Usuwanie wierszy bez ceny
Usunięto 173 wierszy bez ceny

#### KROK 3: Przycięcie wartości odstających

| Zmienna | Średnia przed | Średnia po | Przycięto wartości |
|---------|--------------|------------|-------------------|
| price | 121.61 | 106.43 | 30 (2.1%) |
| minimum_nights | 8.85 | 6.98 | 15 (1.0%) |
| maximum_nights | 411.34 | 411.36 | 15 (1.0%) |

#### KROK 4: Imputacja braków metodą MICE

```r
# Imputacja MICE
mice_vars <- c("bedrooms", "beds", "bathrooms_numeric",
               "review_scores_rating", "review_scores_accuracy",
               "review_scores_cleanliness", "review_scores_checkin",
               "review_scores_communication", "review_scores_location",
               "review_scores_value", "reviews_per_month")

mice_result <- mice(
  df[, mice_vars],
  m = 3,
  method = 'pmm',
  seed = 123,
  printFlag = FALSE
)

df_clean <- complete(mice_result)
```

Czas imputacji MICE: 2.3 sekund
✓ Wszystkie braki zostały uzupełnione!

#### KROK 5: Normalizacja danych
✓ Znormalizowano 8 zmiennych (min-max i z-score)

### Podsumowanie czyszczenia
- Liczba obserwacji przed: 1626
- Liczba obserwacji po: 1453
- Usunięto: 173 (10.6%)
- Liczba kolumn: 94

---

## 10. Analiza zależności

### A. Macierze korelacji (Pearson i Spearman)

![Macierz korelacji Pearsona](https://drive.google.com/uc?export=download&id=1-xIdzgMVfWHyqNuaFvTALhor5K1ikelV)

![Macierz korelacji Spearman](https://drive.google.com/uc?export=download&id=1YITS1pMJEmVhvlvZJ9PTRwXgUwXqDS64)

![Macierz korelacji wszystkich zmiennych](https://drive.google.com/uc?export=download&id=1RD7n1VHewuH8fIq-FO9UXlkO4ABQP9fB)
#### Najsilniejsze korelacje z ceną

| Zmienna | Pearson | Spearman |
|---------|---------|----------|
| accommodates | 0.549 | 0.666 |
| bedrooms | 0.539 | 0.619 |
| bathrooms_numeric | 0.504 | 0.453 |
| beds | 0.488 | 0.565 |
| minimum_nights | 0.096 | 0.141 |
| review_scores_rating | 0.058 | 0.058 |
| availability_365 | -0.021 | -0.057 |
| number_of_reviews | -0.012 | 0.101 |

### Wykresy rozrzutu dla najsilniejszych korelacji
![Scatter plots - cena vs kluczowe zmienne](https://drive.google.com/uc?export=download&id=1E1CI4lc-AFE7pCIImqdum0NU_duT0_rG)


### B. Współczynnik V Craméra dla zmiennych kategorycznych

| Zmienna 1 | Zmienna 2 | V Craméra | Interpretacja |
|-----------|-----------|-----------|---------------|
| room_type | host_response_time | 0.201 | Słaba |
| host_is_superhost | host_response_time | 0.184 | Słaba |
| instant_bookable | host_response_time | 0.137 | Słaba |
| room_type | instant_bookable | 0.109 | Słaba |

### C. Współczynnik R² (kategoryczne → numeryczne)

**TOP 10 najsilniejszych zależności:**

| Zmienna kategoryczna | Zmienna numeryczna | R² | Procent wariancji |
|---------------------|-------------------|-----|-------------------|
| property_type | accommodates | 0.3423 | 34.23% |
| room_type | accommodates | 0.2416 | 24.16% |
| property_type | price | 0.1822 | 18.22% |
| room_type | price | 0.1190 | 11.90% |
| host_is_superhost | review_scores_rating | 0.0603 | 6.03% |
| neighbourhood_cleansed | price | 0.0406 | 4.06% |

![Accommodates według property_type](https://drive.google.com/uc?export=download&id=1trCn9pi1iGeBb41hGdCvLFXg26EU1mbr)

### D. Wykresy par zmiennych
![Macierz wykresów par zmiennych](https://drive.google.com/uc?export=download&id=1DitANChJRPAp6lkGm2F7AIfLJPr3QhUf)


---

## 11. Analiza dodatkowa

### Analiza klastrowa gospodarzy

```r
# Klastrowanie gospodarzy
host_features <- df_clean %>%
  group_by(host_id) %>%
  summarise(
    n_listings = n(),
    avg_price = mean(price, na.rm = TRUE),
    avg_rating = mean(review_scores_rating, na.rm = TRUE),
    .groups = 'drop'
  )

kmeans_result <- kmeans(scale(host_features[, -1]), centers = 4)
host_features$cluster <- as.factor(kmeans_result$cluster)
```

Analizujemy 824 unikalnych gospodarzy

#### Charakterystyka klastrów:

| Nazwa klastra | N | % | Śr. ofert | Śr. cena | Śr. ocena | % Superhostów |
|--------------|---|---|-----------|----------|-----------|---------------|
| Standard | 712 | 86.4% | 1.6 | 88 | 4.83 | 44.5% |
| Premium | 80 | 9.7% | 1.3 | 279 | 4.90 | 38.8% |
| Niska jakość | 27 | 3.3% | 1.5 | 71 | 3.73 | 0% |
| Mega-gracze | 5 | 0.6% | 30.2 | 113 | 4.70 | 20% |

#### Wizualizacja segmentacji

![Segmentacja gospodarzy Airbnb](https://drive.google.com/uc?export=download&id=1De3qzZ7F9pdSmSWVl7hf0SJ8GSNAU6lF)

**Wykres 1:** Widok wszystkich 4 klastrów (skala logarytmiczna osi X)
- **Mega-gracze** (czerwone punkty) - widoczni po prawej stronie, mają 20-50 ofert
- **Premium** (turkusowe punkty) - wysokie ceny (200-350), głównie 1-2 oferty
- **Standard** (fioletowe punkty) - największa grupa, ceny 50-150
- **Niska jakość** (zielone punkty) - nieliczni, niska cena i ocena

![Segmentacja gospodarzy Airbnb](https://drive.google.com/uc?export=download&id=1tAE6-ZupTMRVnRtHVYvPcakuOwDaLIHd)

**Wykres 2:** Zoom na gospodarzy z 1-10 ofertami (99.2% wszystkich gospodarzy)
- Pokazuje tylko 3 klastry (Mega-gracze są poza skalą)
- Wyraźnie widoczna dominacja segmentu Standard
- Segment Premium skupiony w górnej części wykresu
- Segment Niska jakość - pojedyncze czerwone punkty

#### Kluczowe wnioski:
- **86.4% gospodarzy** to segment Standard (średnia cena 88)
- **9.7%** to segment Premium (średnia cena 279)
- Tylko **5 mega-graczy** (0.6%) kontroluje średnio po 30.2 oferty każdy
- **3.3% gospodarzy** ma problemy z jakością (średnia ocena 3.73, brak superhostów)
- Wielkość punktów pokazuje liczbę recenzji - doświadczeni gospodarze mają większe punkty

### Analiza przychodów i obłożenia

```r
# Oszacowanie przychodów
df_revenue <- df_clean %>%
  mutate(
    estimated_nights = pmin(365 - availability_365, number_of_reviews * 3),
    annual_revenue = price * estimated_nights,
    revenue_category = case_when(
      annual_revenue < 5000 ~ "< $5k",
      annual_revenue < 10000 ~ "$5k-10k",
      annual_revenue < 20000 ~ "$10k-20k",
      annual_revenue < 50000 ~ "$20k-50k",
      TRUE ~ "> $50k"
    )
  )
```

#### Szacowane przychody roczne
- Średni przychód: 6,453 CAD
- Mediana przychodu: 2,244 CAD
- Łączny przychód rynku: 9,376,867 CAD

#### Struktura przychodów

| Kategoria przychodu | Liczba ofert | Procent |
|-------------------|--------------|---------|
| < 5k | 924 | 63.6% |
| 5k-10k | 195 | 13.4% |
| 10k-20k | 202 | 13.9% |
| 20k-50k | 125 | 8.6% |
| > 50k | 7 | 0.5% |

![Szacowane roczne przychody według typu pokoju](https://drive.google.com/uc?export=download&id=1_l0K7_4ADrzpZawQ2SEb-Yf4QnxTskfZ)

### Analiza geograficzna

#### TOP 10 dzielnic

| Dzielnica | Liczba ofert | Średnia cena | Mediana ceny | % Entire home | Średnia ocena |
|-----------|--------------|--------------|--------------|---------------|---------------|
| Waverley West | 205 | 98.60 | 76.00 | 40.3% | 4.73 |
| Fort Rouge - East Fort Garry | 199 | 105.00 | 93.00 | 46.9% | 4.71 |
| St. Vital | 167 | 99.50 | 82.00 | 40.3% | 4.75 |
| Daniel McIntyre | 151 | 111.00 | 105.00 | 6.0% | 4.66 |
| Point Douglas | 106 | 119.00 | 97.00 | 6.7% | 4.74 |
| River Heights - Fort Garry | 97 | 141.00 | $113.00 | 6.0% | 4.75 |
| St. Boniface | 94 | 115.00 | 102.00 | 6.7% | 4.74 |
| Transcona | 91 | 76.20 | 70.00 | 40.3% | 4.68 |
| St. James | 78 | 107.00 | 106.00 | 6.7% | 4.81 |
| St. Norbert - Seine River | 73 | 95.10 | 72.00 | 6.0% | 4.81 |

![Średnie ceny według dzielnic](https://drive.google.com/uc?export=download&id=1viJ1qrnFDFL8PFN_ELqihUr3IiKCPlyq)

### Mapa ofert
![Mapa ofert Airbnb w Winnipeg](https://drive.google.com/uc?export=download&id=1Tay5ayxZQb4VoYL9PgbJeG4JPzQEO_w5)

### Koncentracja ofert
![Koncentracja ofert według dzielnic](https://drive.google.com/uc?export=download&id=1ZcajiX1DJO8QKXqpTfVEAKaLXW858U36)

### Analiza trendów czasowych

```r
# Trend liczby nowych gospodarzy
new_hosts_trend <- df_clean %>%
  filter(!is.na(host_since)) %>%
  mutate(host_year = year(host_since)) %>%
  group_by(host_year) %>%
  summarise(
    new_hosts = n_distinct(host_id),
    avg_price = mean(price, na.rm = TRUE),
    .groups = 'drop'
  )
```

![Trend rozwoju rynku Airbnb w Winnipeg](https://drive.google.com/uc?export=download&id=1EtEBQtoL1blR8JBQ4Dz_Pb5V09jCm192)

#### Aktywność ofert
- Aktywne (recenzja w ostatnich 6 mies.): 832 (66.2%)
- Nieaktywne: 424 (33.8%)

### Analiza tekstu nazw ofert

```r
# Analiza słów kluczowych
keywords_analysis <- df_clean %>%
  mutate(
    has_garden = str_detect(tolower(name), "garden"),
    has_luxury = str_detect(tolower(name), "luxury"),
    has_spacious = str_detect(tolower(name), "spacious"),
    has_parking = str_detect(tolower(name), "parking"),
    has_modern = str_detect(tolower(name), "modern"),
    has_downtown = str_detect(tolower(name), "downtown"),
    has_central = str_detect(tolower(name), "central")
  )
```

#### Słowa kluczowe zwiększające cenę

| Słowo kluczowe | Śr. cena z | Śr. cena bez | Premia | % wzrost |
|----------------|-----------|--------------|--------|----------|
| garden | 162.40 | 105.45 | +56.95 | +54.0% |
| luxury | 137.38 | 104.17 | +33.21 | +31.9% |
| spacious | 129.23 | 105.26 | +23.97 | +22.8% |
| parking | 121.65 | 104.76 | +16.90 | +16.1% |
| modern | 119.88 | 104.60 | +15.27 | +14.6% |
| downtown | 112.95 | 105.80 | +7.15 | +6.8% |
| central | 109.42 | 106.30 | +3.12 | +2.9% |

![Słowa kluczowe zwiększające cenę](https://drive.google.com/uc?export=download&id=12CSrAHkGl93rNmH2DjHDpsAcUUwjq-bH)

![Słowa kluczowe zwiększające cenę](https://drive.google.com/uc?export=download&id=1rJCrr-PXz4OEgNT88tgJPY5lnzhxvLmA)

#### Wykorzystanie słów kluczowych

| Słowo kluczowe | Liczba ofert | % użycia | Premia | Wpływ całkowity |
|----------------|--------------|----------|------------|-----------------|
| luxury | 99 | 6.8% | 33.21 | 3287.77 |
| modern | 174 | 12.0% | 15.27 | 2657.66 |
| parking | 144 | 9.9% | 16.90 | 2432.98 |
| spacious | 71 | 4.9% | 23.97 | 1702.05 |
| garden | 25 | 1.7% | 56.95 | 1423.71 |

### Rekomendacje dla gospodarzy

**Najlepsze słowa do użycia w nazwie:**
1. 'garden' - dodaje 57 do ceny (używa tylko 25 ofert)
2. 'luxury' - dodaje 33 do ceny (używa 99 ofert)
3. 'spacious' - dodaje 24 do ceny (używa 71 ofert)

**Przykład optymalnej nazwy:**
'Luxury Garden Suite - Spacious Modern Apartment with Parking'
Potencjalna premia: +146.30

---

## 12. Podsumowanie i wnioski

### Feature Engineering - utworzone zmienne

```r
# Tworzenie nowych zmiennych analitycznych
df_clean <- df_clean %>%
  mutate(
    price_per_person = price / accommodates,
    occupancy_rate = (365 - availability_365) / 365,
    estimated_monthly_revenue = price * (365 - availability_365) / 12,
    is_professional = host_id %in% (host_listings %>%
                                   filter(liczba_ofert > 5) %>%
                                   pull(host_id)),
    size_category = case_when(
      accommodates <= 2 ~ "Small",
      accommodates <= 4 ~ "Medium",
      accommodates <= 6 ~ "Large",
      TRUE ~ "Very Large"
    ),
    host_experience_years = as.numeric(Sys.Date() - host_since) / 365
  )
```

| Nowa zmienna | Średnia | Odch. std. |
|--------------|---------|------------|
| price_per_person | 33.23 | 24.27 |
| occupancy_rate | 40.0% | 36.0% |
| estimated_monthly_revenue | 1,314.05 | 1,629.54 |
| host_experience_years | 5.62 | 3.36 |

### Kluczowe wnioski z analizy

#### 1. Struktura danych
- Po oczyszczeniu danych analizowano 1453 ofert Airbnb w Winnipeg
- Dane zawierają 94 zmiennych (po feature engineering)
- Dominują całe mieszkania/domy (72.4% ofert)
- Brak shared rooms w analizowanym zbiorze

#### 2. Gospodarze i multi-listings
- **59.7% ofert to multi-listings** (biznesowy charakter rynku)
- Top host kontroluje 55 ofert
- 4 największych gospodarzy kontroluje łącznie 149 ofert
- 40.5% ofert prowadzonych przez superhostów

#### 3. Charakterystyka cenowa
- Średnia cena: **106.43 CAD**
- Mediana ceny: **89.00 CAD**
- Rozkład cen jest silnie prawostronnie skośny
- Cena za osobę: średnio 33.23

#### 4. Determinanty ceny
- Najsilniejsze korelacje z ceną:
  - accommodates (r = 0.549)
  - bedrooms (r = 0.539)
  - bathrooms (r = 0.504)
- Property type wyjaśnia 18.22% wariancji ceny
- Słowa kluczowe mogą dodać do 57 do ceny

#### 5. Analiza geograficzna
- Najdroższa dzielnica: Old Kildonan (160)
- Najtańsza dzielnica: Transcona (76)
- Największa koncentracja: Waverley West (205 ofert)

#### 6. Rynek i przychody
- Szacowany roczny obrót: **9.4 mln CAD**
- 63.6% ofert generuje < 5k rocznie
- Tylko 0.5% ofert generuje > 50k rocznie
- Średnie obłożenie: 40.1%

### Rekomendacje

**Dla miasta:**
- Monitorować gospodarzy z >10 ofertami (kontrolują 31% rynku)
- Zwrócić uwagę na 20% ofert z wysokim minimum_nights (potencjalne długoterminowe)

**Dla inwestorów:**
- Optymalna lokalizacja: River Heights-Fort Garry lub Old Kildonan
- Optymalna wielkość: 3-4 osoby, 2 łazienki
- Używać słów: garden, luxury, spacious w nazwie

**Dla platform:**
- 4 segmenty gospodarzy wymagają różnych strategii
- Analiza tekstu ujawnia istotne wzorce cenowe

